# Test script for experiments in paper Sec. 4.2, Supplement Sec. 3, reconstruction from laplacian.

In [3]:
# !pip install plyfile==0.9

In [1]:
# Enable import from parent package
import sys
import os
sys.path.append('siren')

In [2]:
import torch
import modules, utils
import sdf_meshing
import configargparse

In [3]:
from dataclasses import dataclass

@dataclass
class Config:
    # Path to the configuration file
    config_filepath: str = None

    # The root directory for logging
    logging_root: str = './logs'

    # The name of the subdirectory in logging_root where summaries and checkpoints will be saved
    experiment_name: str = 'default_experiment'

    # General training options

    # The batch size for training
    batch_size: int = 1400

    # The path to the checkpoint of the trained model
    checkpoint_path: str = None

    # The type of the model, options are "sine" (all sine activations) and "mixed" (first layer sine, other layers tanh)
    model_type: str = 'sine'

    # The mode of operation, options are "mlp" or "nerf"
    mode: str = 'mlp'

    # The resolution of the output
    resolution: int = 500
    
    device: str = 'cuda:0'

# Initialize the configuration object
opt = Config(
    experiment_name='siren_sdf_baseline',  # Replace this with your experiment name
    checkpoint_path='/app/notebooks/siren_sdf/logs/siren_sdf_baseline/checkpoints/model_current.pth'
)

# Now you can access the values in the config object like this:
print(opt.logging_root)

./logs


In [4]:
class SDFDecoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Define the model.
        if opt.mode == 'mlp':
            self.model = modules.SingleBVPNet(type=opt.model_type, final_layer_factor=1, in_features=3)
        elif opt.mode == 'nerf':
            self.model = modules.SingleBVPNet(type='relu', mode='nerf', final_layer_factor=1, in_features=3)
        self.model.load_state_dict(torch.load(opt.checkpoint_path))
        self.model = self.model.to(opt.device)

    def forward(self, coords):
        model_in = {'coords': coords}
        return self.model(model_in)['model_out']

In [5]:
sdf_decoder = SDFDecoder()

root_path = os.path.join(opt.logging_root, opt.experiment_name)
utils.cond_mkdir(root_path)

sdf_meshing.create_mesh(sdf_decoder, os.path.join(root_path, 'test'), N=opt.resolution)

SingleBVPNet(
  (image_downsampling): ImageDownsampling()
  (net): FCBlock(
    (net): MetaSequential(
      (0): MetaSequential(
        (0): BatchLinear(in_features=3, out_features=256, bias=True)
        (1): Sine()
      )
      (1): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (2): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (3): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (4): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=1, bias=True)
      )
    )
  )
)
0
262144
524288
786432
1048576
1310720
1572864
1835008
2097152
2359296
2621440
2883584
3145728
3407872
3670016
3932160
4194304
4456448
4718592
4980736
5242880
5505024
5767168
6029312
6291456
6553600
6815744
7077888
7340032
7602176
7864320
8126464
8388608
8650752
8912896
9175040
